# Re-create the BBB data 

Any dataset doesn't exist in companies in its raw form, someone has to create it first ... likely from different data sources!

The goal of this project is to re-create the pandas data frame in bbb.pkl
data EXACTLY from its components. Dea steps outlined below:

1. Determine how to load the different file types (use pd.read_pickel, pd.read_csv,
   pd.read_excel, and sqlite3.connect)
2. Determine what data transformations are needed and how the data should be
   combined into a data frame. You MUST name your re-created data frame 'bbb_rec'
3. The final step will be to check that your code produces a data frame
   identical to the pandas data frame in the bbb.pkl file, using pandas' "equals"
   method shown below. 
4. Make sure to style the python code appropriately for easy readable

In [4]:
import pandas as pd
import sqlite3
from datetime import date
import pyrsm as rsm
import urllib.request
from tempfile import NamedTemporaryFile as tmpfile
import os

load the original bbb.pkl data frame from a Dropbox link

In [5]:
bbb_file = tmpfile().name
urllib.request.urlretrieve(
    "https://www.dropbox.com/s/6bulog0ij4pr52o/bbb.pkl?dl=1", bbb_file
)
bbb = pd.read_pickle(bbb_file)

view the data description of the original data to determine
what needs to be re-created

In [6]:
rsm.describe(bbb)

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

Check that the working directory you are using is the same as the location of this file

In [284]:
os.getcwd()

'/home/jovyan/rsm-mgta455-bbb-recreate-data'

load demographics data from bbb_demographics.tsv

In [23]:
bbb_demographics = pd.read_csv('./data/bbb_demographics.tsv',sep = '\t')

bbb_demographics = bbb_demographics.astype({"zip": str})


In [24]:
bbb_demographics

,acctnum,gender,state,zip
0,10001,M,NY,10605
1,10002,M,NY,10960
2,10003,F,PA,19146
3,10004,F,NJ,7016
4,10005,F,NY,10804
...,...,...,...,...
49995,59996,F,NY,11967
49996,59997,F,NJ,8882
49997,59998,M,NJ,7410
49998,59999,M,NJ,7090


In [25]:
bbb_demographics['zip'] = bbb_demographics['zip'].str.zfill(5)

In [26]:
bbb_demographics

,acctnum,gender,state,zip
0,10001,M,NY,10605
1,10002,M,NY,10960
2,10003,F,PA,19146
3,10004,F,NJ,07016
4,10005,F,NY,10804
...,...,...,...,...
49995,59996,F,NY,11967
49996,59997,F,NJ,08882
49997,59998,M,NJ,07410
49998,59999,M,NJ,07090


In [288]:
def cal(row):  
    return row['zip'][:3]


In [27]:
bbb_demographics['zip3'] = bbb_demographics.zip.str[:3]
bbb_demographics.head()

,acctnum,gender,state,zip,zip3
0,10001,M,NY,10605,106
1,10002,M,NY,10960,109
2,10003,F,PA,19146,191
3,10004,F,NJ,07016,070
4,10005,F,NY,10804,108


load nonbook aggregate spending from bbb_nonbook.xls

In [28]:
bbb_nonbook = pd.read_excel('./data/bbb_nonbook.xls')

In [29]:
bbb_nonbook.head()

,acctnum,nonbook
0,10001,248
1,10002,103
2,10003,147
3,10004,257
4,10005,134


load purchase and buy-no-buy information from bbb.sqlite
hint: what data type is "date" in the database?
hint: most systems record dates internally as the number
of days since some origin. You can use the pd.to_datetime
method to convert the number to a date with argument: origin = "1-1-1970"

In [30]:
def db_list_tables(con):
    """Return all table names"""
    cursor = con.cursor()
    cursor.execute("select name from sqlite_master where type='table';")
    return [x[0] for x in cursor.fetchall()]


def db_list_fields(con, tabel):
    """Return all column names for a specified table"""
    cursor = con.cursor()
    cursor.execute(f"select * from {tabel} limit 1;")
    return [name[0] for name in cursor.description]

In [31]:
con = sqlite3.connect("./data/bbb.sqlite")
a = db_list_tables(con)
print(a)
db_list_fields(con,a[0])

['buyer', 'purchase']


['acctnum', 'buyer', 'training']

In [36]:
cursor = con.cursor()
buyer_t = cursor.execute(f"select * from {a[0]} ;")
buyer_t = pd.DataFrame(buyer_t,columns = db_list_fields(con,a[0]) )
buyer_t

,acctnum,buyer,training
0,10001,no,1
1,10002,no,1
2,10003,no,0
3,10004,no,0
4,10005,no,1
...,...,...,...
49995,59996,no,1
49996,59997,no,1
49997,59998,no,1
49998,59999,no,1


In [37]:
purchase_t = cursor.execute(f"select * from {a[1]} ;")
purchase_t = pd.DataFrame(purchase_t,columns = db_list_fields(con,a[1]) )
purchase_t.head()

,acctnum,date,purchase,price
0,10001,13187.0,geog,11.0
1,10001,13191.0,cook,11.0
2,10001,13266.0,youth,9.0
3,10001,13371.0,youth,9.0
4,10001,13457.0,child,10.0


In [46]:
purchase_l = purchase_t.groupby('acctnum')
purchase_l.head()

,acctnum,date,purchase,price
0,10001,13187.0,geog,11.0
1,10001,13191.0,cook,11.0
2,10001,13266.0,youth,9.0
3,10001,13371.0,youth,9.0
4,10001,13457.0,child,10.0
...,...,...,...,...
194506,60000,13794.0,geog,9.0
194507,60000,13811.0,geog,14.0
194508,60000,14330.0,art,10.0
194509,60000,14616.0,child,14.0


In [48]:
purchase_l.price.sum()

acctnum
10001    109.0
10002     35.0
10003     25.0
10004     15.0
10005     15.0
         ...  
59996     15.0
59997     79.0
59998     15.0
59999     98.0
60000     60.0
Name: price, Length: 50000, dtype: float64

In [47]:
purchase_l.date.first()

acctnum
10001    13187.0
10002    13490.0
10003    14099.0
10004    14464.0
10005    14221.0
          ...   
59996    14403.0
59997    13917.0
59998    14586.0
59999    13187.0
60000    13794.0
Name: date, Length: 50000, dtype: float64

In [297]:
def newd(row):
    return pd.to_datetime("1970-01-01") + pd.DateOffset(days=row['date'])

add the zip3 variable

In [298]:
from datetime import timedelta, date

purchase_t['new_date']= purchase_t.apply(lambda row: newd(row), axis=1)
purchase_t.head()

,acctnum,date,purchase,price,new_date
0,10001,13187.0,geog,11.0,2006-02-08
1,10001,13191.0,cook,11.0,2006-02-12
2,10001,13266.0,youth,9.0,2006-04-28
3,10001,13371.0,youth,9.0,2006-08-11
4,10001,13457.0,child,10.0,2006-11-05


In [299]:
d = purchase_t.groupby(['acctnum'])['purchase'].value_counts()
d = d.unstack(level = -1,fill_value = 0)

In [300]:
d = d.reset_index()
d= d.astype({"acctnum": int})

In [301]:
last = purchase_t.groupby('acctnum')['new_date'].max()
first =purchase_t.groupby('acctnum')['new_date'].min()
bbb_demographics.astype({"zip": str})
bbb_demographics.dtypes

acctnum     int64
gender     object
state      object
zip        object
zip3       object
dtype: object

use the following reference date (i.e., "today" for the analysis)

In [302]:
start_date = date(2010, 3, 8)


def diff_months(date1, date2):
    """
    This function calculates the difference in months between
    date1 and date2 when a customer purchased a product
    """
    y = date1.year - date2.year
    m = date1.month - date2.month
    return y * 12 + m

generate the required code below for `first`, `last`, `book`, and `purch`,
and add the purchase frequencies for the different book types
hint: you can use pandas "value_counts" method here
hint: check the help for pandas' `first` and `last` methods

In [303]:
last_l = []
for i in last:
    diff = diff_months(start_date,i)
    last_l.append(diff)

In [304]:
first_l = []
for i in first:
    diff = diff_months(start_date,i)
    first_l.append(diff)

you may find the discussion below of interest at this point
https://stackoverflow.com/questions/65067042/pandas-frequency-of-a-specific-value-per-group

In [305]:
book_l = purchase_t.groupby('acctnum')['price'].sum()


combine the different data frames using pandas' "merge" method

In [306]:
bbb_demographics['first'] = first_l
bbb_demographics['last'] = last_l
bbb_demographics['book'] = list(book_l)
bbb_demographics['nonbook'] = bbb_nonbook['nonbook']
bbb_demographics['total'] = bbb_demographics['nonbook']+bbb_demographics['book']
bbb_demographics['purch'] = list(purchase_l)
bbb_demographics.dtypes

acctnum      int64
gender      object
state       object
zip         object
zip3        object
first        int64
last         int64
book       float64
nonbook      int64
total      float64
purch        int64
dtype: object

In [307]:
bbb_rec = bbb_demographics.merge(d,on = 'acctnum',how = 'inner')
bbb_rec['buyer'] = buyer_t['buyer']
bbb_rec['training'] = buyer_t['training']
bbb_rec = bbb_rec[['acctnum','gender','state','zip','zip3','first','last','book','nonbook','total','purch','child', 'youth', 'cook', 'do_it', 'reference','art','geog','buyer','training']]


In [308]:
bbb_rec = bbb_rec.astype({"acctnum": str,
                'gender' : "category",
                   'state' : "category",
                         'buyer' : "category"})

In [309]:
bbb_rec = bbb_rec.astype({"first": "int32",
                'last' : "int32",
                   'book' : "int32",
                         'nonbook' : "int32",
                          'total' : "int32",
                   'purch' : "int32",
                         'child' : "int32",
                          'youth' : "int32",
                   'cook' : "int32",
                         'do_it' : "int32",
                          'reference' : "int32",
                   'art' : "int32",
                         'geog' : "int32",
                          'training' :"int32"
                          
                         })

check if the columns in bbb and bbb_rec are in the same order
and are of the same type - fix as needed

In [310]:
pd.DataFrame(
    {
        "bbb_names": bbb.columns,
        "bbb_types": bbb.dtypes.astype(str).values,
        "bbb_rec_names": bbb_rec.columns,
        "bbb_rec_types": bbb_rec.dtypes.astype(str),
        "check_names": bbb.columns == bbb_rec.columns,
        "check_types": bbb.dtypes.astype(str).values == bbb_rec.dtypes.astype(str).values
    }
)

,bbb_names,bbb_types,bbb_rec_names,bbb_rec_types,check_names,check_types
acctnum,acctnum,object,acctnum,object,True,True
gender,gender,category,gender,category,True,True
state,state,category,state,category,True,True
zip,zip,object,zip,object,True,True
zip3,zip3,object,zip3,object,True,True
first,first,int32,first,int32,True,True
last,last,int32,last,int32,True,True
book,book,int32,book,int32,True,True
nonbook,nonbook,int32,nonbook,int32,True,True
total,total,int32,total,int32,True,True


add the description as metadata to bbb_rec (see data/bbb_description.txt)
see https://stackoverflow.com/a/40514650/1974918 for more information

In [311]:
bbb.sum()

acctnum      1000110002100031000410005100061000710008100091...
zip          1060510960191460701610804113660646008402074520...
zip3         1061091910701081130640840740800880860100871250...
first                                                  1283446
last                                                    617908
book                                                   2341520
nonbook                                                8074396
total                                                 10415916
purch                                                   194511
child                                                    42723
youth                                                    19549
cook                                                     46830
do_it                                                    23153
reference                                                15612
art                                                      19296
geog                                                   

In [312]:
bbb_rec.sum()

acctnum      1000110002100031000410005100061000710008100091...
zip          1060510960191460701610804113660646008402074520...
zip3         1061091910701081130640840740800880860100871250...
first                                                  1283446
last                                                    617908
book                                                   2341520
nonbook                                                8074396
total                                                 10415916
purch                                                   194511
child                                                    42723
youth                                                    19549
cook                                                     46830
do_it                                                    23153
reference                                                15612
art                                                      19296
geog                                                   

In [313]:
class SubclassedDataFrame(pd.DataFrame):
    # normal properties
    _metadata = ['description']

    @property
    def _constructor(self):
        return SubclassedDataFrame


bbb_rec = SubclassedDataFrame(bbb_rec)


In [314]:
with open('./data/bbb_description.txt') as f:
    contents = f.read()
bbb_rec.description = contents

check that you get the same output for both bbb and bbb_rec

In [315]:
rsm.describe(bbb_rec)
rsm.describe(bbb)

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

# DO NOT EDIT CODE BELOW THIS CELL
# YOUR CODE MUST PASS THE TEST BELOW

In [316]:
test1 = bbb_rec.equals(bbb)
if hasattr(bbb_rec, "description"):
    test2 = bbb_rec.description == bbb.description
else:
    test2 = False

if test1 is True and test2 is True:
    print("Well done! Both tests passed!")
    print("bbb_rec will now be written to the data directory")
    bbb_rec.to_pickle("data/bbb_rec.pkl")
else:
    test = False
    if test1 is False:
        raise Exception(
            """Test of equality of data frames failed.
            Use bbb.dtypes and bbb_rec.dtypes to check
            for differences in types. Check the number
            of mistakes per colmun using, for example,
            (bbb_rec["book"] == bbb["book"]).sum()"""
        )
    if test2 is False:
        raise Exception(
            """Add a description to the bbb_rec data frame.
            Read the description from the txt file in the
            data directory. See
            https://stackoverflow.com/a/40514650/1974918
            for more information"""
        )

Well done! Both tests passed!
bbb_rec will now be written to the data directory
